In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../')

What do I want to achieve, 

1. I want to get a dict of all the places where there are errors with error information (cholecinstanceseg class, cholecT50 class)
2. I want to get a dict of all the accepted frames as well with the key being the video and the values being a list of the frames accepted. 
3. I want to make a visualization of the errors. 


The first two things are to be done together by the way the images are not needed. 

# Get inconsistency_between_t50_and_cholecinstanceg and accepted_frames. 

There would still be action errors by the way, cant solve those without looking through the dataset. 



1.  Do it for a single image, where I know the image_name, ann_names for both dataset. 
2. Then figure out a way to match for a video. 
3. Then figure out a way to match for the whole dataset.  

In [6]:
img_path = '../data/cholecinstanceseg/VID01_t50_full/img_dir/t50_VID01_000000.png'
ann_path_cholecinstanceseg = '../data/cholecinstanceseg/VID01_t50_full/ann_dir/t50_VID01_000000.json' 
ann_path_cholec_T50 = '../data/CholecT50/labels/VID01.json'

In [7]:
import json

In [61]:
from utils.general.read_files import read_from_json
from utils.general.save_files import save_to_json

In [26]:
# # ann_video_01_cholecT50 = read_from_json(ann_path_cholec_T50)
# ann_cholecinstanceseg_img = read_from_json(ann_path_cholecinstanceseg)
# ann_cholecT50_img = ann_video_01_cholecT50['annotations']['0']

Get annotation in CHolecT50

In [24]:
# cholecT50_video_instrument_dict = ann_video_01_cholecT50['categories']['instrument']
# instruments_in_cholecT50_img =  [cholecT50_video_instrument_dict[str(single_ivt_annotation[1])]  for single_ivt_annotation in ann_cholecT50_img]

Get annotations for Cholecinstanceseg

In [40]:
# class_instance_dict = {}

# for ann_shape in ann_cholecinstanceseg_img['shapes']:
#     label = ann_shape['label']
#     instance_id = ann_shape['group_id']
#     if label not in class_instance_dict:
#         class_instance_dict[label] = {instance_id}
#     else:
#         class_instance_dict[label].add(instance_id)    
        
 

In [44]:
# instruments_in_cholecinstanceseg_img = [class_name for class_name, instance_ids in class_instance_dict.items() for _ in instance_ids]

In [42]:
# instruments_in_cholecinstanceseg_img

['grasper']

In [65]:
import os
from os.path import join
from collections import Counter

In [52]:
# from collections import Counter

# img_name = os.path.basename(img_path)
# video_name = 'video_01'

# error_dict_between_both_datasets = {}
# accepted_frames_per_dataset = {}

# accepted_frames_per_dataset[video_name] = [] 
# error_dict_between_both_datasets[video_name] = {} 


# # Check if they have the same elements with the same frequency
# if Counter(instruments_in_cholecinstanceseg_img) == Counter(instruments_in_cholecT50_img):
#     accepted_frames_per_dataset[video_name].append(img_name)
# else:
#     error_dict_between_both_datasets[video_name][img_name] = {
#         'cholecinstanceseg_classes': instruments_in_cholecinstanceseg_img,
#         'cholect50_classes': instruments_in_cholecT50_img  
        
#     } 
#     print("The lists have different elements or frequencies")

In [ ]:
# ann_path_cholecinstanceseg = '../data/cholecinstanceseg/VID01_t50_full/ann_dir/t50_VID01_000000.json' 
# ann_path_cholec_T50 = '../data/CholecT50/labels/VID01.json'

In [92]:
from collections import Counter

list1 = [1, 2, 2, 3]
list2 = [3, 2, 1, 2]

Counter(list1).values()

dict_values([1, 2, 1])

In [93]:
def update_accepted_frames_and_error_dict_per_image(accepted_frames_per_dataset,
                                                    error_dict_between_both_datasets,
                                                    ann_path_cholecinstanceseg,
                                                    vid_ann_path_cholect50,
                                                    ann_id_cholecT50,
                                                    video_name_cholecinstanceseg):
    ann_name = os.path.basename(ann_path_cholecinstanceseg)
    
    
    # instruments in cholecT50
    ann_vid_cholecT50 = read_from_json(vid_ann_path_cholect50)
    ann_cholecT50_img = ann_vid_cholecT50['annotations'][str(ann_id_cholecT50)]
    
    cholecT50_video_instrument_dict = ann_vid_cholecT50['categories']['instrument']
    instruments_in_cholecT50_img =  [cholecT50_video_instrument_dict[str(single_ivt_annotation[1])]  for single_ivt_annotation in ann_cholecT50_img if single_ivt_annotation[1] != -1]
    
    #instruments in cholecinstanceseg
    ann_cholecinstanceseg_img = read_from_json(ann_path_cholecinstanceseg)
    class_instance_dict = {}

    for ann_shape in ann_cholecinstanceseg_img['shapes']:
        label = ann_shape['label']
        instance_id = ann_shape['group_id']
        if label not in class_instance_dict:
            class_instance_dict[label] = {instance_id}
        else:
            class_instance_dict[label].add(instance_id)    
    
    instruments_in_cholecinstanceseg_img = [class_name for class_name, instance_ids in class_instance_dict.items() for _ in instance_ids]


    # Check if they have the same elements with the same frequency
    instruments_in_cholecinstanceseg_img_counter = Counter(instruments_in_cholecinstanceseg_img)
    instruments_in_cholecT50_img_counter = Counter(instruments_in_cholecT50_img)
    if instruments_in_cholecinstanceseg_img_counter == instruments_in_cholecT50_img_counter:
        
        if any(x > 1 for x in instruments_in_cholecinstanceseg_img_counter.values()): # reject for matching error.  
            error_dict_between_both_datasets[video_name_cholecinstanceseg][ann_name] = {
                'reason' : 'matching',
                'cholecinstanceseg_classes': instruments_in_cholecinstanceseg_img,
                'cholect50_classes': instruments_in_cholecT50_img  
                
            } 
            print(f'evaluating {ann_name} - rejected matching.  cholecinstanceseg {instruments_in_cholecinstanceseg_img}, cholect50 {instruments_in_cholecT50_img}')       
            
        else:    
            accepted_frames_per_dataset[video_name_cholecinstanceseg].append(ann_name)
            print(f'evaluating {ann_name} - accepted')
        
        
    else:
        error_dict_between_both_datasets[video_name_cholecinstanceseg][ann_name] = {
            'reason' : 'conflict',
            'cholecinstanceseg_classes': instruments_in_cholecinstanceseg_img,
            'cholect50_classes': instruments_in_cholecT50_img  
            
        } 
        print(f'evaluating {ann_name} - rejected conflict.  cholecinstanceseg {instruments_in_cholecinstanceseg_img}, cholect50 {instruments_in_cholecT50_img}')                                             

OkaY, lets make that into a function, later, first, lets figure out how we match. 

In [94]:
cholecinstanceseg_dataset = '../data/cholecinstanceseg/'
cholect50_dataset = '../data/CholecT50/'

In [114]:
def generate_accepted_frames_and_error_dict(cholecinstanceseg_dataset, cholect50_dataset):
    assert os.path.isdir(cholect50_dataset), 'the cholecT50 dataset path is not a directory' 
    assert os.path.isdir(cholecinstanceseg_dataset), 'the cholecinstanceseg dataset path is not a directory' 
    error_dict_between_both_datasets = {}
    accepted_frames_per_dataset = {}
    for video_name_cholecinstanceseg in os.listdir(cholecinstanceseg_dataset):
        vid_name_cholecT50 =  video_name_cholecinstanceseg.split('_')[0]
        
        vid_ann_dir_cholecinstanceseg = join(cholecinstanceseg_dataset, video_name_cholecinstanceseg, 'ann_dir')
        vid_ann_path_cholect50 = join(cholect50_dataset, 'labels',  f'{vid_name_cholecT50}.json') 
    

        accepted_frames_per_dataset[video_name_cholecinstanceseg] = [] 
        error_dict_between_both_datasets[video_name_cholecinstanceseg] = {} 
        
        for ann_name in os.listdir(vid_ann_dir_cholecinstanceseg):
            ann_id_cholecT50 = int(ann_name.split('.')[0].split('_')[-1])
            ann_path_cholecinstanceseg = join(vid_ann_dir_cholecinstanceseg, ann_name)
            
            update_accepted_frames_and_error_dict_per_image(accepted_frames_per_dataset,
                                                        error_dict_between_both_datasets,
                                                        ann_path_cholecinstanceseg,
                                                        vid_ann_path_cholect50,
                                                        ann_id_cholecT50,
                                                        video_name_cholecinstanceseg)
    
    return  accepted_frames_per_dataset, error_dict_between_both_datasets           

In [115]:
accepted_frames_per_dataset, error_dict_between_both_datasets  = generate_accepted_frames_and_error_dict(cholecinstanceseg_dataset, cholect50_dataset)

evaluating t50_VID01_000000.json - accepted
evaluating t50_VID01_000001.json - accepted
evaluating t50_VID01_000002.json - accepted
evaluating t50_VID01_000003.json - accepted
evaluating t50_VID01_000004.json - rejected conflict.  cholecinstanceseg [], cholect50 ['grasper']
evaluating t50_VID01_000005.json - rejected conflict.  cholecinstanceseg ['grasper'], cholect50 []
evaluating t50_VID01_000006.json - rejected conflict.  cholecinstanceseg ['grasper'], cholect50 []
evaluating t50_VID01_000007.json - rejected conflict.  cholecinstanceseg ['grasper'], cholect50 []
evaluating t50_VID01_000008.json - rejected conflict.  cholecinstanceseg ['grasper'], cholect50 []
evaluating t50_VID01_000009.json - rejected conflict.  cholecinstanceseg ['grasper'], cholect50 []
evaluating t50_VID01_000010.json - accepted
evaluating t50_VID01_000011.json - accepted
evaluating t50_VID01_000012.json - accepted
evaluating t50_VID01_000013.json - accepted
evaluating t50_VID01_000014.json - accepted
evaluating

In [116]:
accepted_frames_per_dataset_save_path = 'outputs/accepted_frames_per_dataset.json'
error_dict_between_both_datasets_save_path = 'outputs/error_dict_between_both_datasets.json'

In [117]:
save_to_json(data=accepted_frames_per_dataset, json_file_path=accepted_frames_per_dataset_save_path)
save_to_json(data=error_dict_between_both_datasets, json_file_path=error_dict_between_both_datasets_save_path)


In [119]:
total_accepted_dataset_length = 0
for vid_name in accepted_frames_per_dataset:
    selected_frames = accepted_frames_per_dataset[vid_name]
    total_accepted_dataset_length += len(selected_frames)

In [120]:
print(total_accepted_dataset_length)    

21443


In [121]:
total_length_of_conflict_errors = 0
total_length_of_matching_errors = 0
total_length_of_removed_frames = 0

for vid_name in error_dict_between_both_datasets:
    info_about_deleted_frames_for_video = error_dict_between_both_datasets[vid_name] 
    total_length_of_removed_frames += len(info_about_deleted_frames_for_video)
    
    removed_frames_due_to_conflict_for_video =  [ann_name for ann_name, ann_info in info_about_deleted_frames_for_video.items() if ann_info['reason'] == 'conflict']
    total_length_of_conflict_errors += len(removed_frames_due_to_conflict_for_video)
    
    removed_frames_due_to_matching_for_video =  [ann_name for ann_name, ann_info in info_about_deleted_frames_for_video.items() if ann_info['reason'] == 'matching']
    total_length_of_matching_errors += len(removed_frames_due_to_matching_for_video)
    
    

In [124]:
print(total_length_of_conflict_errors)
print(total_length_of_matching_errors)
print(total_length_of_removed_frames)

7194
2361
9555


## Get the images